# Post fake data to scammer

on 2020.12.05 I got a text from a UK number (07514 170231):

```
HMRC:Our records show that you have a pending tax rebate. To calculate how much you are owed, visit https://checktax-rebate.com
```

Everything in this is fake:
* Text sent from a private mobile number (or at least it looks like)
* The domain is `com` not `.gov.uk`

When [checking the domain](https://who.is/whois/checktax-rebate.com), it appears this domain was registered the day before. :D Unfortunately all further information is masked by WhoisGuard. 

When checking out the website, it looks OKish, but all the links are dead. 

![title](img/fake_page.png)


So let's submit some fake data into the scammers database. 

What information is needed:
* Full name
* Mother's maiden name
* Date of birth (`DD/MM/YYYY`)
* Address
* City
* Post code
* Phone
* Email
* Name on card
* card number (`####-####-####-####`)
* card expiry (`MM/YY`)
* Security code
* Account number
* sort code (`##-##-##`)



In [103]:
import requests
import json
from faker import Faker
from random import randint, choice
import time

# Initialize a localized faker:
fake = Faker('en_GB')

# Formatters:
fake_card = lambda cn: f'{cn[0:4]}-{cn[4:8]}-{cn[8:12]}-{cn[12:]}'
fake_sort_code = lambda x: f'{x[0:2]}-{x[2:4]}-{x[4:]}'

def generate_name():
    """
    This function ensures the email address is in sync with the name and the email pattern is 
    complicated enough to make it impossible to filter out.
    """
    name = fake.name()
    names = name.lower().split(' ')
    
    # Generating random email:
    first = choice([names[0],names[0][0:2],names[1],names[1][1]])
    last = choice([names[0],names[0][0:2],names[1],names[1][1]])
    
    separator = choice(['','_','.'])
    domain = choice(['google.com','google.co.uk', 'bt.com', 'yahoo.com'])
    
    email = f'{first}{separator}{last}{randint(0,239)}@{domain}'
    
    return (name, email)


def get_fake_payload():
    """
    Returns the string version of the random fake data.
    """
    
    # Building response for the first form:
    form_data = {
        'fullname': name,
        'mothersmaiden': fake.name_female(),
        'email': email,
        'dob': fake.date_of_birth(tzinfo=None, minimum_age=45, maximum_age=95).strftime("%m/%d/%Y"),
        'address1': fake.street_address().replace('\n',' '),
        'address2': '',
        'city': fake.city(),
        'county': fake.county(),
        'postcode': fake.postcode(),
        'phone': '07' + ''.join(["{}".format(randint(0, 9)) for num in range(0, 9)]),
        'cardname': name,
        'cardnumber': fake_card(fake.credit_card_number(card_type='visa')),
        'cardexpiry': fake.credit_card_expire(start='now', end='+3y', date_format='%m/%y'),
        'cardsecurity': fake.credit_card_security_code(),
        'accountnumber': ''.join([str(randint(0,9)) for x in range(8)]),
        'sortcode': fake_sort_code(str(randint(100000,999999)))
    }

    return form_data




In [115]:
url = 'https://checktax-rebate.com/secure/final.action.php'

# Let's submit 1000 fake data:
print('[Info] Sending data ',end='')
while True:
    print('.',end='')
    # Submitting data using fake user agent:
    r = requests.post(url, data = form_data, headers={'user-agent': fake.user_agent()})
#     time.sleep(5)

# 
print('\n[Info] Done')

[Info] Sending data ....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

''